In [9]:
import numpy as np
import pandas as pd
import yaml
import os

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

from keras.models import Sequential, Model
from keras import backend as K
from keras.utils import plot_model
from keras.layers import *
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import (
    Dropout, Input, LSTM, GRU, LeakyReLU,
    Multiply, Add, Lambda, RNN, Bidirectional
)
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.optimizers import Adam

from tensorflow.keras.layers import (
    Input, LSTM, RepeatVector, Conv1D, BatchNormalization,
    Concatenate, Flatten, TimeDistributed, Dense
)

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows


C:\Users\43xqu\AppData\Local\Temp\ipykernel_9644\2797587156.py:35: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


60

In [4]:
def random_index_noise(x, ratio, placeholder=np.nan):
    size = round(len(x) * ratio)
    idx = np.random.randint(low=0, high=len(x), size=size)
    x[idx] = placeholder
    return x

def random_interval_noise(x, min_size, max_size, placeholder=np.nan):
    width = np.random.randint(min_size, max_size)
    where = np.random.randint(0, len(x) - width)
    x[where: where + width] = placeholder
    return x

def apply(X, params):
    '''
    * Lặp hàm `apply_ion_series()` cho tất cả các dòng numpy array 2D

    Chọn ngẫu nhiên 2 lựa chọn:
    1. Random Indices Noise: Cho một `missing value rato`
    2. Random Interval Noise: Cho một độ rộng `width` liên tiếp
    '''

    def apply_on_series(x):
        if np.random.choice([0, 1], p = [1 - params['prob_noise'], params['prob_noise']]):
            nan_ratio = params['missing_value_ratio']
            x = random_index_noise(x, nan_ratio)
        else:
            min_size = params['missing_value_min_size']
            max_size = params['missing_value_max_size']
            x = random_interval_noise(x, min_size, max_size)
        return x

    X = X.values
    X = np.apply_along_axis(apply_on_series, 1, X)
    mask = np.isnan(X)
    return pd.DataFrame(X), pd.DataFrame(mask)

In [25]:
df = pd.read_csv('../data_raw/final.csv')
params = yaml.load(open('../gan/config.yaml'), yaml.Loader)

In [26]:
date = df[['YEAR', 'MO', 'DY']]
df = df.drop(['YEAR', 'MO', 'DY'], axis=1)

In [27]:
df, mask = apply(df, params)

## Model GAN

In [6]:
def adam_optimizer(alpha):
  return Adam(lr=alpha)

def create_generator(adam, timesteps, features, range_matrix, min_matrix):
  #-------------------------------------------------------------
  #units là số lượng phần tử trong sequence (timestep),
  input1 = Input((timesteps, features,))   #Đầu vào 1 là ma trận bị missing value
  mask = Input((timesteps, features,))  #Đầu vào thứ 2 là ma trận mask
  gru = Bidirectional(GRU(units = features, return_sequences=True))(input1)   
  gru1 = GRU(units = features, return_sequences=True)(gru)  #GRU nối với input1 để tạo ra ma trận mới, qua sigmoid nên sẽ nằm trong (0,1)
  gru2 = GRU(units = features, return_sequences=True, activation = "sigmoid")(gru1)
  lambda_ouput = Lambda(lambda x: (x * range_matrix) + min_matrix)(gru2) #Cái này để scale (0,1) thành trong (min,max) nè
  filled = Multiply()([lambda_ouput, mask]) #Nhân ma trận đã được điền khuyết bởi GRU (LSTM, RNN) với ma trận mask (ma trận này chỉ tồn tại giá trị được điền khuyết)
  final = Add()([input1, filled])  #Cuối cùng cộng ma trận ban đầu (input1) với ma trận chỉ tồn tại giá trị được điền khuyết (filled), nhằm tạo ra ma trận đã được điền khuyết

  model = Model(inputs = [input1,mask], outputs = final)
  #-------------------------------------------------------------
  # input1 = Input((timesteps, features,))   #Đầu vào 1 là ma trận bị missing value
  # mask = Input((timesteps, features,))  #Đầu vào thứ 2 là ma trận mask
  # gru = LSTM(units = features, return_sequences=True)(input1)   
  # # gru1 = GRU(units = features, return_sequences=True)(gru)  #GRU nối với input1 để tạo ra ma trận mới, qua sigmoid nên sẽ nằm trong (0,1)
  # # gru2 = GRU(units = features, return_sequences=True, activation = "sigmoid")(gru1)
  # # lambda_ouput = Lambda(lambda x: (x * range_matrix) + min_matrix)(gru2) #Cái này để scale (0,1) thành trong (min,max) nè
  # # filled = Multiply()([lambda_ouput, mask]) #Nhân ma trận đã được điền khuyết bởi GRU (LSTM, RNN) với ma trận mask (ma trận này chỉ tồn tại giá trị được điền khuyết)
  # # final = Add()([input1, filled])  #Cuối cùng cộng ma trận ban đầu (input1) với ma trận chỉ tồn tại giá trị được điền khuyết (filled), nhằm tạo ra ma trận đã được điền khuyết
  
  # model = Model(inputs = [input1,mask], outputs = gru)
  #-------------------------------------------------------------

  model.compile(loss='binary_crossentropy', optimizer=adam)
  
  return model

In [10]:
#Discriminator 
def create_discriminator(adam, timesteps, features):
  input = Input((timesteps, features,))
  # conv1d = Conv1D(filters=20, kernel_size=1)(input)
  gru = Bidirectional(GRU(units = features, return_sequences=True))(input)
  # leaky = LeakyReLU(alpha=0.2)(conv1d)
  leaky = LeakyReLU(alpha=0.2)(gru)
  dropout = Dropout(0.5)(leaky)
  flatten = Flatten()(dropout)
  dense = Dense(1, activation='sigmoid')(flatten) 
  model = Model(inputs = input, outputs = dense)
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

In [11]:
#GAN
def create_gan(discriminator, generator, adam):
  discriminator.trainable=False
  x = generator
  gan_output = discriminator(x.output)
  gan= Model(inputs=x.input, outputs=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer=adam)
  return gan

In [12]:
#Gán nhãn 
def gen_label(size, is_real=True, noise_ratio=0.1):
  if is_real:
    label = np.ones(size,)*1
  else:
    label = np.ones(size,)*0
  return np.squeeze(label)

# tạo input cho Generator
def gen_z_input(batch_size, step, dset, dset_mask):
  return [dset[step*batch_size:(step+1)*batch_size], dset_mask[step*batch_size:(step+1)*batch_size]]

# tạo ra dset fake và label
def gen_fake_batch(generator, batch_size, step, dset, dset_mask):
  z = gen_z_input(batch_size, step, dset, dset_mask)
  fake_dset = generator.predict(z)
  fake_label = gen_label(batch_size, is_real=False)
  return fake_dset, fake_label

# tạo ra dset real và label
def gen_real_batch(batch_size, step, dset):
  real_dset = dset[step*batch_size:(step+1)*batch_size]
  real_label = gen_label(batch_size, is_real=True)
  return real_dset, real_label

In [32]:
def calc_error(pred_matrix, true_matrix, mask_matrix, range_matrix, min_matrix):
  missing_pred = np.multiply(pred_matrix, mask_matrix)
  missing_true = np.multiply(true_matrix, mask_matrix)
  #Normalize
  norm_pred = ((missing_pred - min_matrix)/range_matrix)*mask_matrix
  norm_true = ((missing_true - min_matrix)/range_matrix)*mask_matrix
  num_mask = len(mask_matrix[mask_matrix == True])

  error = np.sum(np.sqrt((norm_pred - norm_true)*(norm_pred - norm_true ))/ num_mask)
  return error

In [33]:
def fill(i, j, Xtest, Xreal, pred, print_full, boundery):
  missingpred = np.multiply(pred[j], Xtest[1][j])
  missingtrue = np.multiply(Xreal[0][j], Xtest[1][j])
  err = calc_error(pred[j], Xreal[0][j], Xtest[1][j])
  #Đã normalize các biến thành (0,1), sau đó tính RMSE
  print("Errors (Normalized): {}".format(err))
  print("------------------------------------------------------------------------------------")
  print("Boundery")
  print(pd.DataFrame(boundery))
  print("------------------------------------------------------------------------------------")
  print('Dữ liệu missing được điền khuyết')
  print(pd.DataFrame(missingpred).round(decimals = 2).replace(0,'-'))
  print("------------------------------------------------------------------------------------")
  print("Dữ liệu missing thực tế")
  print(pd.DataFrame(missingtrue).replace(0,'-') )
  print("------------------------------------------------------------------------------------")
  if(print_full == 1):
    print("Dữ liệu gốc")
    print(pd.DataFrame(Xreal[0][j]))
    print("------------------------------------------------------------------------------------")
    print('Dữ liệu đã được điền')
    print(pd.DataFrame(pred[j].round(decimals = 2)))
    print("------------------------------------------------------------------------------------")
  return err

In [47]:
data = pd.read_csv('../data_raw/final.csv')
df = data[["Temperature","Relative_Humidity","Specific_Humidity","Precipitation","Pressure","Wind_Speed","Wind_Direction"]]

missing_ratio = 0.7

lower_quantile = 0 
upper_quantile = 1

timesteps = 5
features = 7
batch_size = 8

params = {
    "missing_value_ratio": missing_ratio, "prob_noise": 0,
    "missing_value_min_size": 0,"missing_value_max_size": 5, 
    "prob_noise": 0.5, "placeholder_value": -0.1}

missing_df, mask = apply(df, params)
missing = missing_df

boundery = np.array(missing.quantile([lower_quantile, upper_quantile]))
mask = np.array(mask.replace([False,True], [0,1]))
missing = np.array(missing.replace(np.nan, 0))

In [57]:
X = []
X_mask = []
for i in range(len(missing)-timesteps):
  a_missing_sample = missing[i:i+timesteps]
  a_mask_sample = mask[i:i+timesteps]
  X.append(a_missing_sample)
  X_mask.append(a_mask_sample)
X = np.stack(X)
X_mask = np.stack(X_mask)

In [62]:
X_real = []
for i in range(len(np.array(df[["Temperature","Relative_Humidity","Specific_Humidity","Precipitation","Pressure","Wind_Speed","Wind_Direction"]]))-timesteps):
  a_real_sample = np.array(df[["Temperature","Relative_Humidity","Specific_Humidity","Precipitation","Pressure","Wind_Speed","Wind_Direction"]])[i:i+timesteps]
  X_real.append(a_real_sample)
X_real = np.stack(X_real)

In [66]:
X = []
X_mask = []
for i in range(len(missing)-timesteps):
  a_missing_sample = missing[i:i+timesteps]
  a_mask_sample = mask[i:i+timesteps]
  X.append(a_missing_sample)
  X_mask.append(a_mask_sample)
X = np.stack(X)
X_mask = np.stack(X_mask)

X_real = []
for i in range(len(np.array(df[["Temperature","Relative_Humidity","Specific_Humidity","Precipitation","Pressure","Wind_Speed","Wind_Direction"]]))-timesteps):
  a_real_sample = np.array(df[["Temperature","Relative_Humidity","Specific_Humidity","Precipitation","Pressure","Wind_Speed","Wind_Direction"]])[i:i+timesteps]
  X_real.append(a_real_sample)
X_real = np.stack(X_real)

#---------------------------------------------------
X_train = X[0:int(len(X)*0.8)]
X_test = X[int(len(X)*0.8):len(X)]

X_train_real = X[0:int(len(X_real)*0.8)]
X_test_real = X_real[int(len(X_real)*0.8):len(X_real)]

X_mask_train = X_mask[0:int(len(X_mask)*0.8)]
X_mask_test = X_mask[int(len(X_mask)*0.8):len(X_mask)]

#---------------------------------------------------
min_matrix = np.array([boundery[0] for i in range(timesteps)])
max_matrix = np.array([boundery[1] for i in range(timesteps)])
range_matrix = np.array([[boundery[1][i] - boundery[0][i] for i in range(boundery.shape[1])] for j in range(timesteps)])

In [74]:
lrate = 0.0000789 #0.0000789
step_per_epoch = len(X_train)//batch_size - 1

optimizer = adam_optimizer(lrate)
generator = create_generator(optimizer, timesteps, features, range_matrix, min_matrix)
discriminator = create_discriminator(optimizer, timesteps, features)
GAN_model = create_gan(discriminator, generator, optimizer)

epochs= 1
num_epoch = 0